:# Part 1: Detection of Fake / Real Profiles

# Import libraries

In [1]:
pip install pandas-profiling


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/262.6 kB ? eta -:--:--
   ---------------------------------------- 262.6/262.6 kB 8.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
   ---- ----------------------------------- 0.5/4.7 MB 10.7 MB/s eta 0:00:01
   -------- ------------------------------- 1.1/4.7 MB 11.1 MB/s eta 0:00:01
   ------------- -------------------------- 1.6/4.7 MB 11.4 MB/s eta 0:00:01
   ----------------- ---------------------- 2.1/4.7 MB 12.2 MB/s eta 0:00:01
   -------------------- ------------------- 2.5/4.7 MB 11.2 MB/s eta 0:00:01
   ---------------------- ----------------- 2.7/4.7 MB 10.2 MB/s eta 0:00:01
   ------------------------ --------------- 2.9/4.7 MB 9.3 MB/s eta 0:00:01
   -------------------------- ------------- 3.1/4.7 MB 8.7 MB/s eta 0:00:01
   --------------------------- ------------ 3.3/4.7 MB 8.0 MB/s

In [2]:
pip install pandas_profiling

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import pandas_profiling as pp
#from pandas_profiling import ProfileReport
import keras as k
from sklearn.model_selection import train_test_split
import keras
from keras import regularizers
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, Activation
from keras import optimizers
from numpy.random import seed
seed(1)

ModuleNotFoundError: No module named 'keras'

# Read dataset

In [ ]:
df_users = pd.read_csv("realusers.csv")
df_fusers = pd.read_csv("fakeusers.csv")

In [ ]:
df_fusers.shape

In [ ]:
df_users.shape

(1481, 41)

# Add isFake Column

In [ ]:
#for df_users
isNotFake = np.zeros(3474)

#for df_fusers
isFake = np.ones(3351)

In [ ]:
# Check the length of df_fusers
df_fusers_length = len(df_fusers)

# Make sure isFake has the same length as df_fusers
isFake = np.ones(df_fusers_length)

# Assign isFake to df_fusers
df_fusers["isFake"] = isFake

# Check the length of df_users and isNotFake to make sure they match
df_users_length = len(df_users)
isNotFake = np.zeros(df_users_length)

# Assign isNotFake to df_users
df_users["isFake"] = isNotFake


In [ ]:
#for df_users
isNotFake = np.zeros(3474)

#for df_fusers
isFake = np.ones(3351)

# Combine different datasets into one

In [ ]:
df_allUsers = pd.concat([df_fakeusers, df_realusers], ignore_index=True)
df_allUsers.columns = df_allUsers.columns.str.strip()

SyntaxError: ignored

In [ ]:
#to shuffle the whole data
df_allUsers = df_allUsers.sample(frac=1).reset_index(drop=True)

NameError: ignored

In [ ]:
df_allUsers.describe()

In [ ]:
df_allUsers.head()

# Distribution of Data in X and Y

In [ ]:
Y = df_allUsers.isFake

In [ ]:
df_allUsers.drop(["isFake"], axis=1, inplace=True)
X = df_allUsers

In [ ]:
profile = ProfileReport(X, title="Pandas Profiling Report")
profile

In [ ]:
Y.reset_index(drop=True, inplace=True)

In [ ]:
print(Y.shape)

In [ ]:
X.head()

In [ ]:
lang_list = list(enumerate(np.unique(X["lang"])))
lang_dict = {name : i for i, name in lang_list}
X.loc[:, "lang_num"] = X["lang"].map(lambda x: lang_dict[x]).astype(int)

X.drop(["name"], axis=1, inplace=True)

# Feature Selection

In [ ]:
X = X[[
    "statuses_count",
    "followers_count",
    "friends_count",
    "favourites_count",
    "lang_num",
    "listed_count",
    "geo_enabled",
    "profile_use_background_image"
                        ]]

In [ ]:
profile = ProfileReport(X, title="Pandas Profiling Report")
profile

In [ ]:
X = X.replace(np.nan, 0) #To replace the missing boolean values with zeros as it means false

In [ ]:
profile = ProfileReport(X, title="Pandas Profiling Report")
profile

# Import Data

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size=0.8, test_size=0.2, random_state=0)

In [ ]:
print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

# Design Model

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(64, input_dim=32, activation='relu'))
model.add(Dense(64, input_dim=64, activation='relu'))
model.add(Dense(32,input_dim=64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# Compile Model

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Training

In [ ]:
history = model.fit(train_X, train_y,
                    epochs=25,
                    verbose=1,
                    validation_data=(val_X,val_y))

# Testing

In [ ]:
score = model.evaluate(test_X, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Graphs

In [ ]:
# Plot training and validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
axes = plt.gca()
axes.set_xlim([0,14])
axes.set_ylim([0.85,1])
axes.grid(True, which='both')
axes.axhline(y=0.85, color='k')
axes.axvline(x=0, color='k')
axes.axvline(x=14, color='k')
axes.axhline(y=1, color='k')
plt.legend(['Train','Val'], loc='lower right')
plt.show()

In [ ]:
c# Plot training and validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
axes = plt.gca()
axes.set_xlim([0,14])
axes.set_ylim([0,5])
axes.grid(True, which='both')
axes.axhline(y=0, color='k')
axes.axvline(x=0, color='k')
axes.axhline(y=5, color='k')
axes.axvline(x=14, color='k')
plt.legend(['Train','Val'], loc='upper right')
plt.show()

# Prediction

In [ ]:
# Write the index of the test sample to test
prediction = model.predict(test_X[136:137])
prediction = prediction[0]
print('Prediction\n',prediction)
print('\nThresholded output\n',(prediction>0.5)*1)

# Ground truth

In [ ]:
print(test_y[136:137])

# Part 2: Saving, Loading Model
































# Saving

In [ ]:


-# serialize model to JSON
#model_json = model.to_json()

# Write the file name of the model

#with open("model.json", "w") as json_file:
#    json_file.write(model_json)

# serialize weights to HDF5
# Write the file name of the weights

#model.save_weights("model.h5")
#print("Saved model to disk")

# Loading

In [ ]:
# load json and create model

# Write the file name of the model

#json_file = open('model.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#loaded_model = model_from_json(loaded_model_json)

# load weights into new model
# Write the file name of the weights

#loaded_model.load_weights("model.h5")
#print("Loaded model from disk")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# ... (Your existing code)

# Train the model and save training history
history = model.fit(train_X, train_y, epochs=25, verbose=1, validation_data=(val_X, val_y))

# Evaluate the model on the test set
y_pred = (model.predict(test_X) > 0.5).astype(int)

# Calculate metrics
accuracy = accuracy_score(test_y, y_pred)
precision = precision_score(test_y, y_pred)
recall = recall_score(test_y, y_pred)
f1 = f1_score(test_y, y_pred)

# Calculate ROC curve and AUC
y_probs = model.predict(test_X)
fpr, tpr, thresholds = roc_curve(test_y, y_probs)
roc_auc = roc_auc_score(test_y, y_probs)

# Plot training and validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy Over Epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

# Plot training and validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss Over Epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# Print metrics
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")
